# Generate whole file
https://raw.githubusercontent.com/la-rockoteque/Vestigium/refs/heads/main/out/Velum_Cineris;Vestigium_Guide_to_Concord_City.json


In [1]:
import pandas as pd
import inflection

from src.sources import full_source, source, json_source, sources
import src.content.items
import src.content.magic_items
import src.content.feats
import src.content.spells
import src.content.classes
import src.content.subclasses
import src.content.monster
import src.content.conditions
import src.content.features
import src.content.languages
import src.content.species
import src.content.backgrounds
import src.content.dieties
import src.content.diseases

import json
import hashlib
import time


def generate_date_last_modified_hash(doc) -> str:
    if isinstance(doc, dict):
        doc = dict(doc)
        doc.pop("_dateLastModifiedHash", None)

    json_str = json.dumps(
        doc,
        ensure_ascii=False,
        sort_keys=True,
        default=lambda o: list(o) if isinstance(o, set) else o
    )

    md5_hash = hashlib.md5(json_str.encode("utf-8")).hexdigest()
    return md5_hash


current_epoch = time.time()
compendium = {
    "_meta": {
        "sources": sources,
        "unlisted": True,
        "dateAdded": 1743879729,
        "dateLastModified": int(current_epoch),
        "_dateLastModifiedHash": "??????",
        "edition": "classic",
        "currencyConversions": {
            "credit": [{"coin": "credit", "mult": 0.001, "isFallback": True}]
        },
    },
    "$schema": "https://raw.githubusercontent.com/TheGiddyLimit/5etools-utils/master/schema/brew-fast/homebrew.json",
    "race": src.content.species.species_list,
    "class": src.content.classes.classes_list,
    "subclass": src.content.subclasses.subclasses_list,
    "spell": src.content.spells.spells_list,
    "item": src.content.items.items_list + src.content.magic_items.magic_items_list,
    "monster": src.content.monster.monster_list,
    "feat": src.content.feats.feat_list,
    "background": src.content.backgrounds.background_list,
    "condition": src.content.conditions.condition_list,
    "disease": src.content.diseases.disease_list,
    "language": src.content.languages.language_list,
    "deity": src.content.dieties.diety_list,
    "classFeature": src.content.features.features_list,
    "subclassFeature": src.content.features.sub_class_features_list,
    "itemProperty": src.content.items.item_property_list,
}

new_hash = generate_date_last_modified_hash(compendium)
compendium["_meta"]["_dateLastModifiedHash"] = f"{new_hash[:10]}"
# Write the compendium to a JSON file
output_file = f"Velum_Cineris;{inflection.underscore(json_source)}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(
        compendium,
        f,
        ensure_ascii=False,
        indent=4,
        default=lambda o: list(o) if isinstance(o, set) else o
    )

print(f"Export successful! The compendium has been saved as '{output_file}'.")

/Users/rocko/dev/Perso/VestigiumFoundrySpells/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: expected string or bytes-like object, got 'float'